In [1]:
import os

In [2]:
%pwd

'/Users/pradhyumnyadav/Desktop/chest_ctscan_analysis/notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/pradhyumnyadav/Desktop/chest_ctscan_analysis'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import os
import zipfile
import gdown
from ct_scan import logger
from ct_scan.utils.common import get_size
from ct_scan.utils.common import read_yaml, create_directories
# from ct_scan.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH


In [6]:
@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path



In [7]:
class ConfigurationManager:
    def __init__(self):
        self.config = {
            "artifacts_root": "artifacts",
            "data_ingestion": {
                "root_dir": "artifacts/data_ingestion",
                "source_URL": "https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing",
                "local_data_file": "artifacts/data_ingestion/data.zip",
                "unzip_dir": "artifacts/data_ingestion"
            },
            "prepare_base_model": {
                "root_dir": "artifacts/prepare_base_model",
                "base_model_path": "artifacts/prepare_base_model/base_model.h5",
                "updated_base_model_path": "artifacts/prepare_base_model/base_model_updated.h5"
            },
            "training": {
                "root_dir": "artifacts/training",
                "trained_model_path": "artifacts/training/model.h5"
            }
        }
        self.params = {
            "AUGMENTATION": True,
            "IMAGE_SIZE": [224, 224, 3],
            "BATCH_SIZE": 16,
            "INCLUDE_TOP": False,
            "EPOCHS": 10,
            "CLASSES": 2,
            "WEIGHTS": "imagenet",
            "LEARNING_RATE": 0.01
        }

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        data_ingestion_config_data = self.config["data_ingestion"]

        create_directories([data_ingestion_config_data["root_dir"]])

        data_ingestion_config = DataIngestionConfig(
        root_dir=data_ingestion_config_data["root_dir"],
        source_URL=data_ingestion_config_data["source_URL"],
        local_data_file=data_ingestion_config_data["local_data_file"],
        unzip_dir=data_ingestion_config_data["unzip_dir"]
    )

        return data_ingestion_config






In [8]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
     
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [9]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-02-24 14:21:57,387: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-24 14:21:57,388: INFO: 1983665874: Downloading data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1z0mreUtRmR-P-magILsDR3T7M6IkGXtY&confirm=t&uuid=1ac757d5-2d9d-440f-b430-debbc3c4f946
To: /Users/pradhyumnyadav/Desktop/chest_ctscan_analysis/artifacts/data_ingestion/data.zip
100%|██████████| 49.0M/49.0M [00:06<00:00, 7.62MB/s]

[2024-02-24 14:22:05,940: INFO: 1983665874: Downloaded data from https://drive.google.com/file/d/1z0mreUtRmR-P-magILsDR3T7M6IkGXtY/view?usp=sharing into file artifacts/data_ingestion/data.zip]
